### [ Model save & load]
- 2가지 형태 저장
    * 전체 저장
    * 모델의 파라미터만 저장
- 2가지 형태 로딩
    * 전체 저장 모델 파일 => 로딩으로 사용 가능
    * 모델의 파라미터 저장 파일 => 모델 인스턴스 생성 후 층별 파라미터 적용

[1] 모듈 로딩 및 데이터 준비 <hr>

In [152]:
# [1-1] 모듈 로딩
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import util_func as uf                               

In [153]:
# [1-2] 데이터 준비
featureTS = torch.tensor([[1,5], [2, 6], [3, 7], [4, 8], [5, 9]], dtype=torch.float32)
targetTS = torch.tensor([[6], [8], [10], [12], [14]], dtype=torch.float32)

[2] 모델 클래스 정의 <hr>

In [154]:
class Test(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 5)
        self.fc2 = nn.Linear(5, 7)
        self.out = nn.Linear(7, 1)

    def forward(self, x):
        out = F.relu(self.fc1(x))
        out = F.relu(self.fc2(out))
        return self.out(out)

[3] 학습 진행 <hr>

In [155]:
# 설정값들
EPOCHS = 10
BS = 2
LR = 0.1

# 저장 모델 파일명 => 
ALL_MODEL = './AllModel.pt'     # 모델 전체 확장자
WEIGHTS_MODEL = './weights'     # 파라미터 저장 확장자 pth

DEVICE = "cuda" if torch.cuda.is_available() else 'cpu'

In [156]:
# 인스턴스들
model = Test().to(DEVICE)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

dataDS = TensorDataset(featureTS, targetTS)
dataDL = DataLoader(dataDS, batch_size=BS, shuffle=True)

validDS = TensorDataset(featureTS*1.2, targetTS*1.1)
validDL = DataLoader(validDS, batch_size=BS)

In [157]:
for x, y in dataDL:
    print(x, y, sep='\n')
    break

for x, y in validDL:
    print(x, y, sep='\n')
    break

tensor([[4., 8.],
        [5., 9.]])
tensor([[12.],
        [14.]])
tensor([[1.2000, 6.0000],
        [2.4000, 7.2000]])
tensor([[6.6000],
        [8.8000]])


In [ ]:
## 학습진행 ===================================
BEST_LOSS = 100.
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

for epoch in range(EPOCHS):
    #- 학습진행
    train_loss, train_acc = uf.train_one_epoch(model, dataDL, loss_fn, optimizer, DEVICE)

    #- 검증진행
    valid_loss, valid_acc = uf.evaluate(model, validDL, loss_fn, DEVICE)

    #- 모델과 가중치 파일 저장
    if BEST_LOSS > valid_loss:

        #- 모델 전체 저장
        torch.save(model, ALL_MODEL)

        #- 파라미터만 저장
        torch.save(model.state_dict(), f"./{WEIGHTS_MODEL}_{epoch:03}_{valid_loss:.5f}.pth")
        
        #- 기준 loss 점수 업데이트
        BEST_LOSS = valid_loss
    
    #- 학습상태 출력
    print(f"[LOSS] train:valid={train_loss:.7f}:{valid_loss:.7f} ", end=' ') 
    print(f"[ACC] train:valid={train_acc:.7f}:{valid_acc:.7f} ")

[LOSS] train:valid=72.5353831:1.4170122  [ACC] train:valid=0.0000000:0.0000000 
[LOSS] train:valid=12.3396756:1.1004673  [ACC] train:valid=0.0000000:0.0000000 
[LOSS] train:valid=6.0579551:10.4730931  [ACC] train:valid=0.0000000:0.0000000 
[LOSS] train:valid=6.4781755:2.3440032  [ACC] train:valid=0.0000000:0.0000000 
[LOSS] train:valid=3.5790734:4.6457421  [ACC] train:valid=0.0000000:0.0000000 
[LOSS] train:valid=1.2452643:1.5295030  [ACC] train:valid=0.0000000:0.0000000 
[LOSS] train:valid=2.8769956:1.2372892  [ACC] train:valid=0.0000000:0.0000000 
[LOSS] train:valid=2.0786493:6.0532962  [ACC] train:valid=0.0000000:0.0000000 
[LOSS] train:valid=3.0034445:1.2858591  [ACC] train:valid=0.0000000:0.0000000 
[LOSS] train:valid=0.3119909:1.6921558  [ACC] train:valid=0.0000000:0.0000000 


[4] 모델 파일 사용<hr>

In [160]:
# [4-1] 가중치 저장 파일 로딩
params = torch.load("./weights_000_1.41701.pth", weights_only=True)

tModel = Test()
tModel.load_state_dict(params)

<All keys matched successfully>

In [161]:
# [4-2] 전체 모델 저장 파일 로딩
allModel = torch.load(ALL_MODEL, weights_only=False)
allModel

Test(
  (fc1): Linear(in_features=2, out_features=5, bias=True)
  (fc2): Linear(in_features=5, out_features=7, bias=True)
  (out): Linear(in_features=7, out_features=1, bias=True)
)